In [2]:
import numpy

In [3]:
import numba
from numba import guvectorize, float32, float64

In [5]:
@guvectorize([
    (float32[:], float32[:], float32[:]),
    (float64[:], float64[:], float64[:])],
    '(n),(n)->(n)',
)
def _cross(a, b, out):
    """ Cross product of 3d vectors. """
    out[0] = a[1] * b[2] - a[2] * b[1]
    out[1] = a[2] * b[0] - a[0] * b[2]
    out[2] = a[0] * b[1] - a[1] * b[0]

def cross(a, b, out=None):
    a, b = numpy.broadcast_arrays(a, b)
    
    if out is None:
        out = numpy.empty_like(a)
    
    assert a.shape[-1] == 3
    assert out.shape == a.shape
    
    _cross(a, b, out)
    
    return out

In [13]:
s = numpy.linspace(0, 2*numpy.pi, 1000)

In [14]:
ta = numpy.zeros([1000, 3], dtype=float)
ta[:,0] = numpy.sin(s)
ta[:,1] = numpy.cos(s)

In [15]:
%%timeit
cross(numpy.expand_dims(ta, 0), numpy.expand_dims(ta, 1))

3.64 ms ± 47.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [16]:
%%timeit
numpy.cross(numpy.expand_dims(ta, 0), numpy.expand_dims(ta, 1))

27.1 ms ± 550 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
